In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df

,image_id,filename,label
0,1,1.jpg,2
1,2,2.jpg,4
2,3,3.jpg,2
3,4,4.jpg,3
4,5,5.jpg,5
...,...,...,...
7195,7196,7196.jpg,4
7196,7197,7197.jpg,4
7197,7198,7198.jpg,4
7198,7199,7199.jpg,2


In [4]:
df.isnull().sum()

image_id    0
filename    0
label       0
dtype: int64

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [6]:
import os

# Get the current working directory
current_directory = os.getcwd()

print(f"Current working directory: {current_directory}")

Current working directory: /Users/kan3k1/Projects/FRAUD_CLAIMS/train


In [7]:
train_dir = '/Users/kan3k1/Projects/FRAUD_CLAIMS/train/images'
train_csv = '/Users/kan3k1/Projects/FRAUD_CLAIMS/train/train.csv'

In [8]:
train_dir

'/Users/kan3k1/Projects/FRAUD_CLAIMS/train/images'

In [9]:
train_csv

'/Users/kan3k1/Projects/FRAUD_CLAIMS/train/train.csv'

In [10]:
train_df = pd.read_csv(train_csv)

In [11]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  
)

train_generator = datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),  
    batch_size=32,
    class_mode='raw',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    subset='validation'
)

Found 5760 validated image filenames.
Found 1440 validated image filenames.


In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
  5/180 [..............................] - ETA: 26:56 - loss: 26.0736 - accuracy: 0.3063